# PDF 문서 분석 및 텍스트 추출

이 노트북은 PDF 문서에서 제목과 본문을 추출하고 구조화하는 코드를 포함합니다.

## 주요 기능
1. 제목 정제 (TitleCleaner)
2. 제목 매칭 (TitleMatcher)
3. PDF 처리 (PDFProcessor)

## 필요한 라이브러리 설치
```bash
pip install PyMuPDF
```

In [ ]:
import fitz  # PyMuPDF
import os
import json
import re
from typing import List, Optional, Tuple, Dict, Any

## 1. TitleCleaner 클래스
제목 텍스트를 정제하는 클래스입니다. 다양한 패턴의 제목을 정규화하고 불필요한 요소를 제거합니다.

In [ ]:
class TitleCleaner:
    """제목 정제를 담당하는 클래스"""
    
    @staticmethod
    def clean_title(text: str) -> str:
        """제목 텍스트를 정제하는 메서드"""
        # >>>> 로 시작하는 부분 제거
        text = re.sub(r'^>>>>\s*', '', text)
        
        # 앞의 숫자(01., 1. 등)로 시작하는 경우만 제거
        if not (re.match(r'^\d{4}년', text) or re.match(r'^제\s*\d+\s*[장절]', text)):
            text = re.sub(r'^\d+\.?\s*', '', text)
        
        text = TitleCleaner._normalize_year_patterns(text)
        text = TitleCleaner._remove_page_numbers(text)
        text = TitleCleaner._process_middle_dots(text)
        text = TitleCleaner._normalize_date_patterns(text)
        text = TitleCleaner._normalize_spaces(text)
        
        return text.strip()
    
    @staticmethod
    def _normalize_year_patterns(text: str) -> str:
        """연도 패턴을 정규화"""
        patterns = [
            (r'(\d{4})\s*년도', r'\1년'),  # 2024년도 -> 2024년
            (r'(\d{4})\s*년(?!\s*적용)', r'\1년'),  # 2024 년 -> 2024년
            (r'\'(\d{2})년(?!\s*적용)', r'20\1년'),  # '24년 -> 2024년
            (r'^(\d{2})년(?!\s*적용)', r'20\1년'),  # 24년 -> 2024년
            (r'(\d{4})\s*[~∼]\s*(\d{4})\s*년', r'\1-\2년'),  # 2024~2025년 -> 2024-2025년
            (r'(\d{4})\.(\d{1,2})\.(\d{1,2})', r'\1년 \2월 \3일')  # 2024.1.1 -> 2024년 1월 1일
        ]
        
        for pattern, replacement in patterns:
            text = re.sub(pattern, replacement, text)
        return text
    
    @staticmethod
    def _remove_page_numbers(text: str) -> str:
        """페이지 번호와 관련 구분자 제거"""
        patterns = [
            r'\s*[･·ㆍ∙]{2,}\s*\d+\s*$',  # 중간점 + 숫자
            r'\s*\.{3,}\s*\d+$',  # 점(...) + 숫자
            r'\s*\d+$',  # 숫자
            r'\s*\.{3,}\s*'  # 남은 점(...)
        ]
        
        for pattern in patterns:
            text = re.sub(pattern, '', text)
        return text
    
    @staticmethod
    def _process_middle_dots(text: str) -> str:
        """중간점 처리"""
        # 단일 중간점(∙) 제거 (장/절 번호 사이)
        text = re.sub(r'\s*[∙]\s*', ' ', text)
        
        # 의미있는 중간점 처리
        def replace_middle_dot(match):
            original = match.group(0)
            return ' ･ ' if ' ' in original else '･'
            
        return re.sub(r'\s*[･·ㆍ]\s*', replace_middle_dot, text)
    
    @staticmethod
    def _normalize_date_patterns(text: str) -> str:
        """날짜 패턴 정규화"""
        text = re.sub(r'(\d{1,2})\s*[월]\s*', r'\1월 ', text)
        text = re.sub(r'(\d{1,2})\s*[일]\s*', r'\1일 ', text)
        return text
    
    @staticmethod
    def _normalize_spaces(text: str) -> str:
        """공백 정규화"""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\(\s*\)', '', text)  # 빈 괄호 제거
        return text.strip()

## 2. TitleMatcher 클래스
텍스트가 제목과 매칭되는지 확인하는 클래스입니다. 다양한 매칭 방법을 제공합니다.

In [ ]:
class TitleMatcher:
    """제목 매칭을 담당하는 클래스"""
    
    def __init__(self, cleaner: TitleCleaner):
        self.cleaner = cleaner
    
    def is_title_match(self, text: str, title: str) -> bool:
        """텍스트가 제목과 매칭되는지 확인"""
        cleaned_text = self.cleaner.clean_title(text).lower()
        cleaned_title = self.cleaner.clean_title(title).lower()
        
        if not cleaned_text or not cleaned_title:
            return False
            
        # 정확히 일치하는 경우
        if cleaned_text == cleaned_title:
            return True
            
        # 페이지 번호와 점(...)을 제거한 후 비교
        text_without_page = re.sub(r'\s*[.·･]{2,}\s*\d+\s*$', '', cleaned_text)
        title_without_page = re.sub(r'\s*[.·･]{2,}\s*\d+\s*$', '', cleaned_title)
        
        if text_without_page == title_without_page:
            return True
            
        # 번호 패턴 제거 후 비교
        text_without_number = re.sub(r'^(?:\d+\.)?\s*', '', text_without_page)
        title_without_number = re.sub(r'^(?:\d+\.)?\s*', '', title_without_page)
        
        if text_without_number == title_without_number:
            return True
            
        return self._check_partial_match(text_without_number, title_without_number)
    
    def _check_partial_match(self, text: str, title: str) -> bool:
        """부분 매칭 확인"""
        # 특수문자로 구분된 단어들을 분리
        title_words = self._split_with_delimiters(title)
        text_words = self._split_with_delimiters(text)
        
        if len(title_words) > 1:
            # 의미있는 단어만 선택 (2글자 이상)
            main_words = [w for w in title_words if len(w) > 1]
            if not main_words:
                return False
                
            # 매칭되는 단어 수 계산
            matched_words = sum(1 for word in main_words if 
                              any(word in text_part for text_part in text_words))
            
            # 70% 이상 매칭되면 True
            return matched_words >= len(main_words) * 0.7
            
        # 단일 단어인 경우 정확히 일치해야 함
        return text == title
    
    @staticmethod
    def _split_with_delimiters(text: str) -> List[str]:
        """특수문자로 구분된 단어 분리"""
        # 특수문자로 분리
        parts = re.split(r'([-･·ㆍ])', text)
        words = []
        i = 0
        while i < len(parts):
            if i + 2 < len(parts) and parts[i+1] in ['-', '･', '·', 'ㆍ']:
                words.append(parts[i] + parts[i+1] + parts[i+2])
                i += 3
            else:
                if parts[i].strip():
                    words.append(parts[i].strip())
                i += 1
        return [w for w in words if w and not w in ['-', '･', '·', 'ㆍ']]

## 3. PDFProcessor 클래스
PDF 문서를 처리하고 목차를 추출하는 클래스입니다.

In [ ]:
class PDFProcessor:
    """PDF 처리를 담당하는 클래스"""
    
    def __init__(self, cleaner: TitleCleaner, matcher: TitleMatcher):
        self.cleaner = cleaner
        self.matcher = matcher
    
    def extract_toc(self, doc: fitz.Document) -> List[str]:
        """목차 추출"""
        toc = doc.get_toc()
        if toc:
            return [title.strip() for _, title, _ in toc]
            
        return self._extract_toc_manually(doc)
    
    def _extract_toc_manually(self, doc: fitz.Document) -> List[str]:
        """수동으로 목차 추출"""
        titles = []
        found_titles = set()
        current_title_lines = []
        
        for page in doc:
            self._process_page_for_toc(page, titles, found_titles, current_title_lines)
            
        # 마지막 제목 처리
        if current_title_lines and self._is_page_number_format(" ".join(current_title_lines)):
            self._process_current_title(current_title_lines, titles, found_titles)
            
        return titles
    
    def _process_page_for_toc(self, page: fitz.Page, titles: List[str], 
                             found_titles: set, current_title_lines: List[str]) -> None:
        """페이지 단위 목차 처리"""
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" not in block:
                continue
                
            for line in block["lines"]:
                line_text = self._extract_line_text(line)
                if not line_text:
                    continue
                    
                self._process_line_for_toc(line_text, titles, found_titles, current_title_lines)
    
    @staticmethod
    def _extract_line_text(line: dict) -> str:
        """라인에서 텍스트 추출"""
        line_text = ""
        for span in line["spans"]:
            span_text = span["text"].strip()
            if span_text:
                if line_text and not (line_text.endswith('･') or line_text.endswith('·') or 
                                    line_text.endswith('ㆍ') or line_text.endswith('-') or
                                    line_text.endswith('.')):
                    line_text += " "
                line_text += span_text
        return line_text
    
    def _process_line_for_toc(self, line_text: str, titles: List[str], 
                             found_titles: set, current_title_lines: List[str]) -> None:
        """목차용 라인 처리"""
        if self._is_title_start(line_text):
            if current_title_lines:
                self._process_current_title(current_title_lines, titles, found_titles)
                current_title_lines.clear()
            current_title_lines.append(line_text)
            if self._is_page_number_format(line_text):
                self._process_current_title(current_title_lines, titles, found_titles)
                current_title_lines.clear()
        elif current_title_lines:
            current_title_lines.append(line_text)
            if self._is_page_number_format(line_text):
                self._process_current_title(current_title_lines, titles, found_titles)
                current_title_lines.clear()
    
    def _process_current_title(self, current_title_lines: List[str], 
                             titles: List[str], found_titles: set) -> None:
        """현재 제목 처리"""
        if current_title_lines:
            full_title = " ".join(line.strip() for line in current_title_lines)
            cleaned_text = self.cleaner.clean_title(full_title)
            if cleaned_text and cleaned_text not in found_titles:
                titles.append(full_title)
                found_titles.add(cleaned_text)
    
    @staticmethod
    def _is_title_start(text: str) -> bool:
        """제목 시작 패턴 확인"""
        text = text.strip()
        return (text.startswith('[') or
                re.match(r'^\d+\.', text) or
                re.match(r'^제\s*\d+\s*[장절]', text))

## 4. 사용 예제
위의 클래스들을 사용하여 PDF 문서를 처리하는 예제입니다.

In [ ]:
def process_pdf(pdf_path: str, output_path: str):
    """PDF 파일을 처리하여 JSON으로 저장"""
    # 클래스 인스턴스 생성
    cleaner = TitleCleaner()
    matcher = TitleMatcher(cleaner)
    processor = PDFProcessor(cleaner, matcher)
    
    # PDF 문서 열기
    doc = fitz.open(pdf_path)
    
    try:
        # 목차 추출
        titles = processor.extract_toc(doc)
        
        # 결과를 JSON으로 저장
        result = {"titles": titles}
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
            
        print(f"처리가 완료되어 {output_path}에 저장되었습니다.")
        
    finally:
        doc.close()

# 사용 예제
if __name__ == "__main__":
    pdf_path = "sample.pdf"
    output_path = "output.json"
    process_pdf(pdf_path, output_path)